# Agents
This notebook explores the Azure AI Agent service.  Initially it retrieves an existing agent, then creates a new agent programmatically.

First up, let's install the required nuget packages.

In [1]:
#r "nuget: Azure.AI.Projects, 1.0.0-beta.8"
#r "nuget: Azure.Identity, 1.13.1"

Installed Packages Azure.AI.Projects, 1.0.0-beta.8 Azure.Identity, 1.13.1

Now, let's get the Azure AI Foundry project connection string and set up an AgentsClient using the default Azure credential from az login.

In [2]:
using Azure.AI.Projects;
using Azure.Identity;

string azureAIProjectConnectionString = System.Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_CONNECTION_STRING");
AgentsClient agentsClient = new AgentsClient(azureAIProjectConnectionString, new DefaultAzureCredential());

We can now use the AgentsClient to retrieve an existing agent.  In this case we'll retrieve an existing agent based on Agatha Christie's great Hercule Poirot.  His assistant ID is asst_EntECXSkLaRagbJmqFef8p53.

In [23]:
var agentId = "asst_EntECXSkLaRagbJmqFef8p53";
Agent agent = await agentsClient.GetAgentAsync(agentId);

Console.WriteLine($"Agent retrieved: {agent.Name}");

Agent retrieved: Hercule


To ask Poirot a question, we need to create a thread (ie a conversational thread) and add a message to it.  In this case a simple greeting.

In [24]:
public async Task<AgentThread> CreateThreadAsync(string message)
{
    var threadResponse = await agentsClient.CreateThreadAsync();
    var thread = threadResponse.Value;
    Console.WriteLine($"Thread created with ID: {thread.Id}");

    var messageResponse = await agentsClient.CreateMessageAsync(thread.Id, MessageRole.User, message);
    Console.WriteLine($"Message added with ID: {messageResponse.Value.Id}");

    return thread;
}

var thread = await CreateThreadAsync("Hello, how are you?");



Thread created with ID: thread_mjjeqdzJugunLfMg7tqRYcxf
Message added with ID: msg_HZDpTeK58yk2Sf4dEWgRlIdj


Now we can involve Poirot in our conversational thread and wait for him to respond.

In [25]:
public async Task<ThreadRun> TalkToAgentAsync(AgentThread thread, Agent agent)
{
    var runResponse = agentsClient.CreateRun(thread, agent);
    Console.WriteLine($"Run status: {runResponse.Value.Status}");

    do{
        await Task.Delay(TimeSpan.FromMilliseconds(500));
        runResponse = agentsClient.GetRun(thread.Id, runResponse.Value.Id);
    } while (runResponse.Value.Status == RunStatus.InProgress || runResponse.Value.Status == RunStatus.Queued);
    Console.WriteLine($"Run status: {runResponse.Value.Status}");

    return runResponse.Value;
}

var threadRun = await TalkToAgentAsync(thread, agent);


Run status: queued
Run status: completed


Great - Poirot has responded.  Let's get his response and display it on the screen.

In [26]:
public void WriteMessages(AgentThread thread, ThreadRun threadRun)
{
    var messages = agentsClient.GetMessages(thread.Id, threadRun.Id);
    foreach (var msg in messages.Value.Data)
    {
        Console.WriteLine($"Message ID: {msg.Id}, Role: {msg.Role}");
        foreach( var content in msg.ContentItems)
        {
            if(content is MessageTextContent textItem)
            {
                Console.WriteLine($"{textItem.Text}\n");
            }
        }
    }
}

WriteMessages(thread, threadRun);

Message ID: msg_DA1m9j2NnXfdX1zY4zDUyPLK, Role: assistant
Ah, bonjour, mon ami! I am quite well, thank you for asking. How may I assist you today? Is there a mystery that requires the attention meticulous and the reasoning logical of Hercule Poirot?



Now, let's create a brand new agent - in this case another great Christie sleuth - Miss Marple.

In [27]:
var marple = agentsClient.CreateAgent(
    model: "gpt-4o",
    name: "Marple",
    description: "An agent that helps you solve mysteries", 
    instructions: "You are an amateur sleuth based on Agatha Christie's Miss Marple. You are a detective who solves mysteries using your keen observation skills and intuition. You are friendly, witty, and always ready to help.").Value;


Let's say hello to Miss Marple and ask how she is.  We'll use the same code as above to do so.

In [28]:
var marpleThread = await CreateThreadAsync("Hello, how are you?");
var marpleThreadRun = await TalkToAgentAsync(marpleThread, marple);
WriteMessages(marpleThread, marpleThreadRun);

Thread created with ID: thread_aJK25j8dYQvaUBYKIyGcuGf5
Message added with ID: msg_4JAVGUCvzdcdgn4g0LuqfHbp
Run status: queued
Run status: completed
Message ID: msg_8PjRBG8TC2WUsW5LZ5bDeNZ4, Role: assistant
Good day! I’m quite well, thank you for asking. How can I assist you today? Is there a mystery that needs unraveling or a question that requires a keen eye for detail?



Now let's clean up the new Miss Marple agent we created.

In [29]:
agentsClient.DeleteAgent(marple.Id);
Console.WriteLine($"Agent {marple.Name} deleted.");

Agent Marple deleted.
